In [1]:
import tensorflow as tf

2026-01-16 03:51:39.883728: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-16 03:51:40.949940: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import sys
sys.path.append(r'/esi/project/niwa00018/rampaln/PUBLICATIONS/2026/CORDEX_ML_TF/WGAN')

In [4]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow
import xarray as xr
from dask.diagnostics import ProgressBar
from tensorflow.keras.callbacks import Callback
import numpy as np
import pandas as pd
import sys
import os
sys.path.append(os.getcwd())
from src.layers import res_block_initial, \
    BicubicUpSampling2D,upsample, conv_block,decoder_noise,down_block,up_block



In [6]:
test_tensor = tf.keras.layers.Input((16,16,64))

In [106]:
output= down_block(test_tensor , num_filters[2], 3, i =1, attn_type = "self")

In [43]:
resize_output =[128, 128]
input_size = [16,16]
num_filters = [32, 64, 128, 256]
num_channels =8

In [15]:

def unet(input_size, resize_output, num_filters, num_channels, num_classes,
                          final_activation = 'linear', orog_predictor = True):

    """

    :param input_size:
    :param resize_output: high-resolution target fields, for CORDEX-ML-Bench this is 128x128.
    :param num_filters: a list consisting of number of filters in individual layrs [32, 64, 128] ...
    :param kernel_size: not used
    :param num_channels:
    :param num_classes:
    :param resize:
    :param final_activation:
    :return:
    """
    if orog_predictor:
        input_topography_fields = tf.keras.layers.Input(shape=[resize_output[0], resize_output[1], 1])

    large_scale_fields = tf.keras.layers.Input(
        shape=[input_size[0], input_size[1], num_channels],
        name='spatial_input'
    )
    time_of_year = tf.keras.layers.Input(shape=[1], name='time_input')
    conditioned_large_scale_fields = TimeFilmLayer(
        num_channels=num_channels,
        hidden_dim=128,
        use_sinusoidal=True,
        max_period=365,
        name='time_film'
    )([large_scale_fields, time_of_year])

    # High-resolution inputs
    if orog_predictor:
        x, temp1 = down_block(input_topography_fields, num_filters[0], kernel_size=3, i =0, attn_type ="None") # 64, 64
        x, temp2 = down_block(x, num_filters[1], kernel_size=5, i =1, attn_type ="None") # 32, 32
        x, temp3 = down_block(x, num_filters[2], kernel_size=5, i =2, attn_type ="None") # 16, 16

    # Film conditioned low resolution inputs (16 x 16 inputs)
    x1 = res_block_initial(conditioned_large_scale_fields, [num_filters[2]], 5, [1, 1], f"test2123434", attn_type ="channel")
    x1 = res_block_initial(x1, [num_filters[2]*2], 3, [1, 1], f"test21233434", attn_type ="channel")
    x1 = res_block_initial(x1, [64], 5, [1, 1], f"test1", attn_type ="None")
    x1 = res_block_initial(x1, [128], 5, [1, 1], f"test2", attn_type ="None")

    # Merge the inputs together
    if orog_predictor:
        concat_scales = tf.keras.layers.Concatenate(-1)([x1, x])
    else:
        concat_scales = x1

    x = res_block_initial(concat_scales, [256], 3, [1, 1], f"test3", attn_type ="cbam")
    # decode
    if orog_predictor:
        x = up_block(x, temp3, kernel_size=5, filters = num_filters[2], i =0, concat = True, attn_type ="channel")
        x = up_block(x, temp2, kernel_size=5, filters = num_filters[1], i =2, concat = True, attn_type ="channel")
        x = up_block(x, temp1, kernel_size=3, filters = num_filters[0], i =3, concat = True, attn_type ="self")
    else:
        x = up_block(x, [], kernel_size=5, filters = num_filters[2], i =0, concat = False, attn_type ="channel")
        x = up_block(x, [], kernel_size=5, filters = num_filters[1], i =2, concat = False, attn_type ="channel")
        x = up_block(x, [], kernel_size=3, filters = num_filters[0], i =3, concat = False, attn_type ="self")

    # Final output convolutions
    output = x
    output = res_block_initial(output, [32], 5, [1, 1], "output_convbbb12347", attn_type ="None")
    output = tf.keras.layers.Conv2D(num_classes, 1, activation=final_activation, padding ='same')(output)
    if orog_predictor:
        input_layers = [large_scale_fields, input_topography_fields, time_of_year]
    else:
        input_layers = [large_scale_fields, time_of_year]
    model = tf.keras.models.Model(input_layers, output, name='unet')
    model.summary()
    return model


In [17]:
gan_model = unet([16, 16], [128, 128], [32, 32, 64, 128], 8, 1,
                          final_activation = tf.keras.layers.LeakyReLU(1), orog_predictor = True)

Model: "unet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ spatial_input       │ (None, 16, 16, 8) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_film           │ (None, 16, 16, 8) │     18,960 │ spatial_input[0]… │
│ (TimeFilmLayer)     │                   │            │ time_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_227 (Conv2D) │ (None, 16, 16,    │     12,864 │ time_film[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_148     │ (None, 16, 16,    │          0 │ conv2d_227[0][0]  │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_229 (Conv2D) │ (None, 16, 16,    │        576 │ time_film[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_228 (Conv2D) │ (None, 16, 16,    │    102,464 │ leaky_re_lu_148[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_67 (Add)        │ (None, 16, 16,    │          0 │ conv2d_229[0][0], │
│                     │ 64)               │            │ conv2d_228[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ se_block_20         │ (None, 16, 16,    │      1,096 │ add_67[0][0]      │
│ (SEBlock)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_218 (Conv2D) │ (None, 128, 128,  │        320 │ input_layer_7[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_149     │ (None, 16, 16,    │          0 │ se_block_20[0][0] │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_142     │ (None, 128, 128,  │          0 │ conv2d_218[0][0]  │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_230 (Conv2D) │ (None, 16, 16,    │     73,856 │ leaky_re_lu_149[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_220 (Conv2D) │ (None, 128, 128,  │         64 │ input_layer_7[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_219 (Conv2D) │ (None, 128, 128,  │      9,248 │ leaky_re_lu_142[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_150     │ (None, 16, 16,    │          0 │ conv2d_230[0][0]

 Total params: 3,468,417 (13.23 MB)

 Trainable params: 3,468,417 (13.23 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
critical_model = critic([128, 128,1],[16,16, 8
                          ], use_bn=False,
                            use_dropout=False, use_bias=True,
                            high_resolution_feature_channels=(16, 32, 64, 128))

In [11]:
critical_model.summary()

Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_76 (Conv2D)  │ (None, 128, 128,  │        416 │ input_layer_4[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_52      │ (None, 128, 128,  │          0 │ conv2d_76[0][0]   │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_78 (Conv2D)  │ (None, 128, 128,  │         32 │ input_layer_4[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_77 (Conv2D)  │ (None, 128, 128,  │      6,416 │ leaky_re_lu_52[0… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_22 (Add)        │ (None, 128, 128,  │          0 │ conv2d_78[0][0],  │
│                     │ 16)               │            │ conv2d_77[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention2d_2  │ (None, 128, 128,  │        613 │ add_22[0][0]      │
│ (SelfAttention2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_53      │ (None, 128, 128,  │          0 │ self_attention2d… │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_83 (Conv2D)  │ (None, 128, 128,  │      2,320 │ leaky_re_lu_53[0… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_54      │ (None, 128, 128,  │          0 │ conv2d_83[0][0]   │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_85 (Conv2D)  │ (None, 128, 128,  │        272 │ leaky_re_lu_53[0… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_84 (Conv2D)  │ (None, 128, 128,  │      2,320 │ leaky_re_lu_54[0… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_23 (Add)        │ (None, 128, 128,  │          0 │ conv2d_85[0][0],  │
│                     │ 16)               │            │ conv2d_84[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ se_block_6          │ (None, 128, 128,  │         82 │ add_23[0][0]      │
│ (SEBlock)           │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_55      │ (None, 128, 128,  │          0 │ se_block_6[0][0]  │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_86 (Conv2D)  │ (None, 64, 64,    │     12,832 │ leaky_re_lu_55[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_57      │ (None, 64, 64,    │          0 │ conv2d_86[0][0] 

 Total params: 1,460,012 (5.57 MB)

 Trainable params: 1,460,012 (5.57 MB)

 Non-trainable params: 0 (0.00 B)